In [60]:
import os
import numpy as np
import pandas as pd
import selfies
import torch
import chemistry_vae_selfies
import data_loader
import chemestry_perceptron as cp
import torch.nn as nn
import yaml

In [47]:
my_file = pd.read_csv('./datasets/QM9listprops.csv', index_col=None)##The file you want to train on, should contain SMILES reps, latent space reps and properties
my_file.dropna()

list_of_properties = []


for i in range(len(my_file)):

    DipoleIn = my_file['dipole_moment'][i]
    GapIn = my_file['energy_gap'][i]
    SMILESCodeIn = my_file['smiles'][i]
    list_of_properties.append([SMILESCodeIn, GapIn, DipoleIn])

In [11]:
# load the encoder and decoder models from the VAE training

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

folder_path = "./datasets/"
file_name = "SelectedSMILES_QM9.txt"

full_path = folder_path + file_name

selfies_list, selfies_alphabet, largest_selfies_len, smiles_list, smiles_alphabet, largest_smiles_len = chemistry_vae_selfies.get_selfie_and_smiles_encodings_for_dataset(full_path)

#selfies_alphabet = ['[#Branch2]', '[Ring2]', '[Branch2]', '[=Branch2]', '[O]', '[=O]', '[=C]', '[=N]', '[#Branch1]', '[=Branch1]', '[nop]', '[N]', '[Branch1]', '[F]', '[#C]', '[#N]', '[Ring1]', '[C]']

###

    # define source file location
file_to_load =  "./saved_models_RNN/"
    # training file name encoder
training_file_nameE = "300/E"
    # training file name decoder
training_file_nameD = "300/D"
    # load data
    #load_data_trained = file_to_load + training_file_nameE
    # Alphabet has 18 letters, largest molecule is 21 letters. (build this as an output function later ... )
largest_selfies_len_dataset = largest_selfies_len
largest_smiles_len_dataset = largest_smiles_len

    #in_dimension = len(selfies_alphabet)*largest_selfies_len
in_dimension = len(smiles_alphabet)*largest_smiles_len

    # load the trained encoder
vae_encoder = torch.load(file_to_load + training_file_nameE) #, map_location=torch.device(device="cpu"))
    #print(vae_encoder)

    # load the trained decoder
vae_decoder = torch.load(file_to_load + training_file_nameD) #, map_location=torch.device(device="cpu"))
    #print(vae_decoder)


selfies_alphabet = ['[#Branch2]', '[Ring2]', '[Branch2]', '[=Branch2]', '[O]', '[=O]', '[=C]', '[=N]', '[#Branch1]', '[=Branch1]', '[nop]', '[N]', '[Branch1]', '[F]', '[#C]', '[#N]', '[Ring1]', '[C]']

###

--> Translating SMILES to SELFIES...
Finished translating SMILES to SELFIES.
selfies aplhabet: ['[Branch2]', '[N]', '[=O]', '[=C]', '[Ring1]', '[Branch1]', '[Ring2]', '[#C]', '[O]', '[=N]', '[=Branch1]', '[#Branch1]', '[=Branch2]', '[nop]', '[F]', '[#N]', '[#Branch2]', '[C]']
smiles aplhabet: ['4', '2', 'F', '=', '1', 'C', ')', '3', '5', '#', 'N', '(', 'O', ' ']


In [14]:
# select the molecules without pentabonds
smiles_rep, latent_rep, props_used = cp.filter(list_of_properties, largest_selfies_len_dataset, selfies_alphabet, vae_encoder, vae_decoder)

vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector skipped
vector ski

In [51]:
# convert to torch tensors and generate latent space vectors

input_dim = 25 ### Size of the latent space

latent_space_vectors_valid = [item[0].detach().squeeze(0) for item in latent_rep]
properties_training = [torch.tensor(property_vector) for property_vector in props_used]  # this will convert lists to tensors

train_size = round(len(latent_space_vectors_valid)*0.8)

energy_gaps = []
for i in range(len(properties_training)):
    energy_gaps.append(props_used[i][0])

latents_train = latent_space_vectors_valid[:train_size]
latents_test = latent_space_vectors_valid[train_size:]

props_train = energy_gaps[:train_size]
props_test = energy_gaps[train_size:]

train_latent_space_tensor, test_latent_space_tensor, train_properties_tensor, test_properties_tensor = cp.make_tensors(latents_train, latents_test, props_train, props_test)

train_latent_space_tensor = train_latent_space_tensor.view(-1, input_dim)
test_latent_space_tensor = test_latent_space_tensor.view(-1, input_dim)

In [77]:
if os.path.exists("settings_perceptron.yml"):
    settings = yaml.safe_load(open("settings_perceptron.yml", "r"))
else:
    print("Expected a file settings_perceptron.yml but didn't find it.")
    
print('--> Acquiring data...')
# set the parameters manualy
hidden_dim = settings['architecture']['hidden_dim'] 
prop_hidden_dim = settings['architecture']['prop_hidden_dim']
prop_pred_activation =  settings['architecture']['prop_pred_activation']
prop_pred_dropout =  settings['architecture']['prop_pred_dropout']
prop_pred_depth = settings['architecture']['prop_pred_depth']
prop_growth_factor =  settings['architecture']['prop_growth_factor']
lr_input= settings['training']['lr']
epochs = settings['training']['num_epochs']

--> Acquiring data...


In [78]:
# Instantiate the model with hyperparameters
model = cp.PropertyRegressionModel(input_dim, hidden_dim, prop_hidden_dim, prop_pred_activation, prop_pred_dropout, prop_pred_depth, prop_growth_factor)

# Define your loss function (e.g., Mean Squared Error)
loss_function = nn.MSELoss()

# Define your optimizer (e.g., Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_input)

###

for epoch in range(epochs):
        optimizer.zero_grad()
        predictions = model(train_latent_space_tensor)
        loss = loss_function(predictions.squeeze(), train_properties_tensor)
        loss.backward()
        optimizer.step()



        ###

        with torch.no_grad():
                test_predictions = model(test_latent_space_tensor)
                test_loss = loss_function(test_predictions.squeeze(), test_properties_tensor)
                print(f"Test Loss: {test_loss.item():.4f}")

        ###


        # Generate predictions for the test set
        y_pred = test_predictions
        y_test = props_test

        mse, mae, r2 = cp.stats(y_pred, y_test)


Test Loss: 62.1787
Test Loss: 0.2090
Test Loss: 0.0328
Test Loss: 0.0499
Test Loss: 0.0219
Test Loss: 0.0117
Test Loss: 0.0112
Test Loss: 0.0063
Test Loss: 0.0082
Test Loss: 0.0110
Test Loss: 0.0104
Test Loss: 0.0078
Test Loss: 0.0138
Test Loss: 0.0098
Test Loss: 0.0111
Test Loss: 0.0123
Test Loss: 0.0089
Test Loss: 0.0122
Test Loss: 0.0075
Test Loss: 0.0101
Test Loss: 0.0056
Test Loss: 0.0077
Test Loss: 0.0038


[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

In [79]:
r2

-2.876308033040707